# Clearing address from dataframe origin - Tratando base de dados dos endereços do dataframe original. 

In [ ]:
!pip install geopandas

In [51]:
!pip install shapely

In [2]:
import matplotlib 
import pandas as pd
import geopandas as gpd

In [56]:
from shapely.geometry import Point, Polygon  

In [70]:
from geopandas import geoseries 

- csv file generated from building.ipynb notebook. -- Arquivo csv gerado a partir do notebook building.ipynb. 

In [3]:
ppi_address_ajust = '/content/address_ppi.csv'
ppi_address_ajust = pd.read_csv(ppi_address_ajust, encoding='ISO-8859-1')
ppi_address_ajust.head()

,Unnamed: 0,Empreendimento,UF
0,0,Aeroporto de Salvador/ BA,BA
1,1,Aeroporto de Florianópolis/ SC,SC
2,2,Aeroporto de Fortaleza/ CE,CE
3,3,Aeroporto de Porto Alegre/ RS,RS
4,4,Aeroporto de Maceió/ AL,AL


In [4]:
ppi_address_ajust.rename(columns= {'Unnamed: 0': 'Rows_place'}, inplace=True )

In [5]:
ppi_address_ajust

,Rows_place,Empreendimento,UF
0,0,Aeroporto de Salvador/ BA,BA
1,1,Aeroporto de Florianópolis/ SC,SC
2,2,Aeroporto de Fortaleza/ CE,CE
3,3,Aeroporto de Porto Alegre/ RS,RS
4,4,Aeroporto de Maceió/ AL,AL
...,...,...,...
186,186,"LOTE 9, Atendimento elétrico à região sudeste ...",AM/PA
187,187,"LOTE 10, Expansão do sistema de transmissão pa...",RJ/ES
188,188,Rodovia BR-153/282/470/SC e SC-412 \n,SC
189,189,EF-118 Ferrovia Rio-Vitória (Trecho de Nova Ig...,RJ/ES


In [6]:
ppi_address_ajust.dtypes

Rows_place         int64
Empreendimento    object
UF                object
dtype: object

In [ ]:
ppi_address_ajust.values

- 'brazil_states'

In [8]:
gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'Brazil')

,geometry,address
0,POINT (-38.34109 -12.90422),"Aeroporto, Travessa Santos Dumont, Lauro de Fr..."
1,POINT (-48.53533 -27.68058),"Rodovia Acesso ao Aeroporto, Carianos, Ribeirã..."
2,POINT (-38.53549 -3.77626),Aeroporto Internacional de Fortaleza - Pinto M...
3,POINT (-51.18287 -29.98773),"Aeroporto, Acesso a sede Trensurb, Humaitá, Po..."
4,POINT (-35.71050 -9.66392),"Aeroporto, Rua Jangadeiros Alagoanos, Pajuçara..."
...,...,...
186,GEOMETRYCOLLECTION EMPTY,None
187,GEOMETRYCOLLECTION EMPTY,None
188,GEOMETRYCOLLECTION EMPTY,None
189,GEOMETRYCOLLECTION EMPTY,None


In [9]:
ppi_address_ajust['Geom'] = gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'brazil_states')['geometry']

In [13]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

In [14]:
ppi_address_ajust

,Rows_place,Empreendimento,UF,Geom
0,0,Aeroporto de Salvador/ BA,BA,POINT (-38.34109 -12.90422)
1,1,Aeroporto de Florianópolis/ SC,SC,POINT (-48.53533 -27.68058)
2,2,Aeroporto de Fortaleza/ CE,CE,POINT (-38.53549 -3.77626)
3,3,Aeroporto de Porto Alegre/ RS,RS,POINT (-51.18287 -29.98773)
4,4,Aeroporto de Maceió/ AL,AL,POINT (-35.71050 -9.66392)
5,5,Aeroporto de João Pessoa/ PB,PB,POINT (-34.94622 -7.15926)
6,6,Aeroporto de Aracaju/ SE,SE,POINT (-37.07603 -10.98281)
7,7,Aeroporto de Juazeiro do Norte/ CE,CE,POINT (-39.26783 -7.21970)
8,8,Aeroporto de Campina Grande/ PB,PB,POINT (-35.89610 -7.27016)
9,9,Aeroporto de Recife/ PE,PE,POINT (-34.91461 -8.13418)


In [79]:
ppi_point = ppi_address_ajust['Geom']

In [ ]:
ppi_point

In [ ]:
ppi_point.isna()

In [111]:
# ppi_geo_empty = ppi_address_ajust.loc[ppi_address_ajust['Geom'] == 'GEOMETRYCOLLECTION EMPTY']

In [115]:
# ppi_geo_empty = ppi_address_ajust.loc[ppi_point == 'GEOMETRYCOLLECTION EMPTY']

In [119]:
ppi_point.loc

- To remember codes: 

In [ ]:
# ppi_address_ajust.index.values
# ppi_address_ajust.Geom.values
# type(ppi_address_ajust.Geom.values)